https://towardsdatascience.com/how-to-download-images-from-google-photos-using-python-and-photos-library-api-6f9c1e60a3f3

In [47]:
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
#from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request
import requests

class GooglePhotosApi:
    def __init__(self,
                 api_name = 'photoslibrary',
                 client_secret_file= r'./credentials/client_secret.json',
                 api_version = 'v1',
                 scopes = ['https://www.googleapis.com/auth/photoslibrary']):
        '''
        Args:
            client_secret_file: string, location where the requested credentials are saved
            api_version: string, the version of the service
            api_name: string, name of the api e.g."docs","photoslibrary",...
            api_version: version of the api

        Return:
            service:
        '''

        self.api_name = api_name
        self.client_secret_file = client_secret_file
        self.api_version = api_version
        self.scopes = scopes
        self.cred_pickle_file = f'./credentials/token_{self.api_name}_{self.api_version}.pickle'

        self.cred = None

    def run_local_server(self):
        # is checking if there is already a pickle file with relevant credentials
        if os.path.exists(self.cred_pickle_file):
            with open(self.cred_pickle_file, 'rb') as token:
                self.cred = pickle.load(token)

        # if there is no pickle file with stored credentials, create one using google_auth_oauthlib.flow
        if not self.cred or not self.cred.valid:
            if self.cred and self.cred.expired and self.cred.refresh_token:
                self.cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.client_secret_file, self.scopes)
                self.cred = flow.run_local_server()

            with open(self.cred_pickle_file, 'wb') as token:
                pickle.dump(self.cred, token)
        
        return self.cred

In [48]:
# initialize photos api and create service
google_photos_api = GooglePhotosApi()
creds = google_photos_api.run_local_server()

In [49]:
#Listing all Albums:
def getAlbum(album_name):
    album_url = "https://photoslibrary.googleapis.com/v1/sharedAlbums"
    headers = {
            'content-type': 'application/json',
            'Authorization': 'Bearer {}'.format(creds.token)
        }

    try:
        res = requests.request("GET",album_url, headers=headers)
    except:
        print('Request error')
    json_result = res.json()
    print(res.status_code)

    for i in range(0, len(json_result["sharedAlbums"])) :
        if json_result["sharedAlbums"][i]['title'] == album_name :
            return json_result["sharedAlbums"][i]['id']
    print("Album not found.")
    return None

album_id = getAlbum("Sync Flickr")
album_id

200


'AF474M2R3WSrxhJsjobPgQuTOxKdwmEVhaQkOmmHqBzxZQw8q2lMK_k0658rBKYVxqQZMUFUetpHYMHGjX9q_zYqy9cKOlDu8w'

The following API call lists all the items to be downloaded. if this number exceeds 100, then data will be missed.

In [50]:
import json
import requests

def get_response_from_medium_api(year, month, day, album_id=None):
    url = 'https://photoslibrary.googleapis.com/v1/mediaItems:search'
    
    payload = {
                "filters": {
                  "dateFilter": {
                    "dates": [
                      {
                        "day": day,
                        "month": month,
                        "year": year
                      }
                    ]
                  }
                }
              }
    headers = {
        'content-type': 'application/json',
        'Authorization': 'Bearer {}'.format(creds.token)
    }
    
    try:
        res = requests.request("POST", url, data=json.dumps(payload), headers=headers)
    except:
        print('Request error') 
    
    return(res)

In [51]:
def list_of_media_items(year, month, day, album_id, media_items_df):
    '''
    Args:
        year, month, day, album_id: day for the filter of the API call 
        media_items_df: existing data frame with all find media items so far
    Return:
        media_items_df: media items data frame extended by the articles found for the specified tag
        items_df: media items uploaded on specified date
    '''

    items_list_df = pd.DataFrame()
    
    # create request for specified date
    response = get_response_from_medium_api(year, month, day, album_id)

    try:
        for item in response.json()['mediaItems']:
            items_df = pd.DataFrame(item)
            items_df = items_df.rename(columns={"mediaMetadata": "creationTime"})
            items_df.set_index('creationTime')
            items_df = items_df[items_df.index == 'creationTime']

            #append the existing media_items data frame
            items_list_df = pd.concat([items_list_df, items_df])
            media_items_df = pd.concat([media_items_df, items_df])
    
    except:
        print(response.text)

    return(items_list_df, media_items_df)

media_items_df = pd.DataFrame()
list_of_media_items(2022, 12, 21, album_id, media_items_df)

(                                                             id  \
 creationTime  AF474M1fX_l2CbhuU3t6K2CajGS4JSy602RpE7pYLmWhHd...   
 creationTime  AF474M0PQAr0pVLrONijGAW6P14GA2qyyA3LD9JTypKRfu...   
 
                                                      productUrl  \
 creationTime  https://photos.google.com/lr/photo/AF474M1fX_l...   
 creationTime  https://photos.google.com/lr/photo/AF474M0PQAr...   
 
                                                         baseUrl    mimeType  \
 creationTime  https://lh3.googleusercontent.com/lr/ANt_8_bhU...  image/jpeg   
 creationTime  https://lh3.googleusercontent.com/lr/ANt_8_aDA...  image/jpeg   
 
                       creationTime                       filename  
 creationTime  2022-12-21T16:47:16Z  PXL_20221221_164716420.MP.jpg  
 creationTime  2022-12-21T16:47:13Z     PXL_20221221_164713657.jpg  ,
                                                              id  \
 creationTime  AF474M1fX_l2CbhuU3t6K2CajGS4JSy602RpE7pYLmWhHd...   
 c

In [52]:
# getting data for a aspecific date up to now
import pandas as pd
from datetime import date, timedelta, datetime
import requests

# Images should only be downloaded if they are not already available in downloads
# Herefor the following code snippet, creates a list with all filenames in the /downloads/ folder
files_list = os.listdir(r'./downloads')
files_list_df = pd.DataFrame(files_list)
files_list_df = files_list_df.rename(columns={0: "filename"})
files_list_df.head(2)

# create a list with all dates between start date and today
sdate = date(2022,12,28)   # start date
edate = date.today()
date_list = pd.date_range(sdate,edate-timedelta(days=1),freq='d')
print(date_list)

# name of the album
album_id = getAlbum("Sync Flickr")

media_items_df = pd.DataFrame()

for date in date_list:
    
    # get a list with all media items for specified date (year, month, day)
    items_df, media_items_df = list_of_media_items(year = date.year, month = date.month, day = date.day, album_id=album_id,  media_items_df = media_items_df)

    if len(items_df) > 0:
        # full outer join of items_df and files_list_df, the result is a list of items of the given 
        #day that have not been downloaded yet
        items_not_yet_downloaded_df = pd.merge(items_df, files_list_df,on='filename',how='left')
        items_not_yet_downloaded_df.head(2)

        # download all items in items_not_yet_downloaded
        for index, item in items_not_yet_downloaded_df.iterrows():
            url = item.baseUrl
            response = requests.get(url)

            file_name = item.filename
            destination_folder = './downloads/'

            with open(os.path.join(destination_folder, file_name), 'wb') as f:
                f.write(response.content)
                f.close()
                
        print(f'Downloaded items for date: {date.year} / {date.month} / {date.day}')
    else:
        print(f'No media items found for date: {date.year} / {date.month} / {date.day}')
            
#save a list of all media items to a csv file
current_datetime = str(datetime.now())
filename = f'item-list-{current_datetime}.csv'

#save a list with all items in specified time frame
media_items_df.to_csv(f'./media_items_list/{filename}', index=True)

DatetimeIndex(['2022-12-28', '2022-12-29', '2022-12-30', '2022-12-31',
               '2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06', '2023-01-07', '2023-01-08'],
              dtype='datetime64[ns]', freq='D')
200
Downloaded items for date: 2022 / 12 / 28
Downloaded items for date: 2022 / 12 / 29
{}

No media items found for date: 2022 / 12 / 30
{}

No media items found for date: 2022 / 12 / 31
Downloaded items for date: 2023 / 1 / 1
{}

No media items found for date: 2023 / 1 / 2
{}

No media items found for date: 2023 / 1 / 3
{}

No media items found for date: 2023 / 1 / 4
{}

No media items found for date: 2023 / 1 / 5
{}

No media items found for date: 2023 / 1 / 6
{}

No media items found for date: 2023 / 1 / 7
{}

No media items found for date: 2023 / 1 / 8
